# Embedded Simulator in Braket Jobs

このノートブックでは、Braket Jobs の Embedded Simulator を紹介します。Embedded Simulator とは、ジョブインスタンス (アルゴリズムスクリプトを実行する計算リソース) 上で動作するローカルシミュレータのことです。一方、SV1, DM1, TN1 などの[オンデマンドシミュレータ](https://docs.aws.amazon.com/braket/latest/developerguide/braket-devices.html#braket-simulator-sv1)は、Amazon Braket によるオンデマンドの専用計算リソース上で量子回路の結果を計算するものです。量子・古典ハイブリッドワークロードは通常、量子回路の実行と変分パラメータの最適化の繰り返しで構成されています。Embedded Simulator を使用することで、全ての計算を同じ環境で行うことができます。これにより、最適化アルゴリズムは、Embedded Simulator が持つさまざまな機能を使うことができます。例えば、PennyLane を介して、サポートされているシミュレータに[随伴法や誤差逆伝播法](https://pennylane.readthedocs.io/en/stable/introduction/interfaces.html#simulation-based-differentiation) などの高度な勾配計算手法を活用することができます。これらの方法は、波動関数の中間状態へのアクセスに依存しており、Amazon Braket のオンデマンドシミュレータや QPU ではサポートされていません。さらに、Jobs の [Bring Your Own Container (BYOC)](https://docs.aws.amazon.com/braket/latest/developerguide/braket-jobs-byoc.html) 機能により、ユーザーはオープンソースシミュレータや独自のシミュレーションツールを使うこともできます。

手動で設定した[Amazon EC2 インスタンス](https://aws.amazon.com/ec2/)や、ユーザーのローカル環境で実行するローカルシミュレータとは異なり、Hybrid Jobs はお客様に代わって計算資源を管理します。ジョブインスタンスは、ジョブの作成時に自動的に起動し、ジョブが終了すると終了するため、お客様は使用したリソースに対してのみ料金を支払います。お客様は複数のジョブを同時に投入することで、ハイパーパラメーターの最適化などの実験を加速させることができます。また、ジョブ作成時に選択するデバイスを変更することで、Embedded Simulator から QPU を含む他の Amazon Braket デバイスに切り替えることもできます。

## Embedded Simulator に用いるデバイスの指定

普通、ジョブを作成する際には、`AwsQuantumJob.create()` の `device` 引数にオンデマンドシミュレータか QPU の ARN (Amazon Resource Name) を指定します。しかし Embedded Simulator を使う際は、代わりに `device` 引数に <br> 
`device = "local:<provider>/<simulator_name>"` <br> という形式の文字列を指定します。
`<provider>` と `<simulator_name>` は、英数字、`_`、`-`、`.` のみから構成される必要があります。例えば、[Braket-Pennylane プラグイン](https://github.com/aws/amazon-braket-pennylane-plugin-python)を使って Embedded Simulator を使うには、次のように記述します。

In [3]:
device = "local:braket/braket.local.qubit"

アルゴリズムスクリプトは、環境変数 `"AMZN_BRAKET_DEVICE_ARN"` を通じて、文字列 `device` にアクセスできます。 Braket では、`<provider>` と `<simulator_name>` にロジックを実行しません。お客様は、アルゴリズムスクリプト内で `<provider>` と `<simulator_name>` を指定し、任意のシミュレータを作成することができます。このノートでは、Braket-PennyLane プラグインを通してシミュレータを使用します。アルゴリズムスクリプトの一部として、ヘルパー関数 `get_device()` を用意し、`device` 変数をパースして PennyLane `qml.device` をセットアップします。

In [4]:
!cat qaoa/utils.py

import pennylane as qml
import os

def get_device(n_wires):
    device_string = os.environ["AMZN_BRAKET_DEVICE_ARN"]
    device_prefix = device_string.split(":")[0]

    if device_prefix=="local":
        prefix, device_name = device_string.split("/")
        device = qml.device(device_name, 
                            wires=n_wires, 
                            custom_decomps={"MultiRZ": qml.MultiRZ.compute_decomposition})
        print("Using local simulator: ", device.name)
    else:
        device = qml.device('braket.aws.qubit', 
                             device_arn=device_string, 
                             s3_destination_folder=None,
                             wires=n_wires,
                             parallel=True,
                             max_parallel=30)
        print("Using AWS managed device: ", device.name)
        
    return device

## Embedded Simulator を用いたジョブの実行
今回の例では、Embedded Simulator 上で Hybrid Jobs を使い、グラフの全結合ノードの最大集合を求める Max-Clique 問題に対して QAOA アルゴリズムを実行します。詳細については、[グラフ最適化ノートブック](https://github.com/aws/amazon-braket-examples/blob/main/examples/pennylane/2_Graph_optimization_with_QAOA/2_Graph_optimization_with_QAOA.ipynb)を参照ください。今回使う QAOA スクリプトは、[qaoa_algorithm.py](https://github.com/aws/amazon-braket-examples/blob/main/examples/hybrid_jobs/4_Embedded_simulators_in_Braket_Jobs/qaoa/qaoa_algorithm.py) で定義されています。このアルゴリズムでは、`n_nodes` や `n_edges` といった Max-Clique 問題の設定に関連するハイパーパラメータと、反復回数やステップ数といった学習そのものに関連するパラメータが必要です。

In [5]:
hyperparameters={
    "n_nodes": 6, 
    "n_edges": 8, 
    "n_layers": 3,
    "iterations": 10,
    "stepsize": 0.1,
    "seed": 42,
    "diff_method": "parameter-shift",
}

アルゴリズムは PennyLane で書かれているため、対応するコンテナを選択する必要があります。PennyLane を含む構成済みのコンテナとして、PyTorch コンテナと TensorFlow コンテナの 2 つがあります。設定済みのコンテナについては [開発者ガイド](https://docs.aws.amazon.com/braket/latest/developerguide/braket-jobs-script-environment.html) を参照ください。それでは、PyTorch コンテナを実行してみましょう。

In [6]:
from braket.aws import AwsSession
from braket.jobs.image_uris import Framework, retrieve_image

region = AwsSession().region
image_uri = retrieve_image(Framework.PL_PYTORCH, region)
print(image_uri)

292282985366.dkr.ecr.us-east-1.amazonaws.com/amazon-braket-pytorch-jobs:1.13.1-gpu-py39-cu117-ubuntu20.04


ハイブリッドジョブで Embedded Simulator を使用する場合、回路はジョブインスタンス上で実行されます。たくさんの量子ビットを持つ回路のシミュレーションなど、より大きな計算リソースを必要とするシミュレーションでは、多くのコア数や大容量メモリを持つジョブインスタンスを選択する必要があります。lightning.gpu のような GPUベースのシミュレータを使うには、GPU インスタンスを選択する必要があります。インスタンスを設定するには、`InstanceConfig` 引数を使います。利用可能なインスタンスタイプは、[開発者ガイド](https://docs.aws.amazon.com/braket/latest/APIReference/API_InstanceConfig.html) に記載されています。ここでは、汎用的なインスタンスタイプ (ml.m5.large) を使用することにしましょう。

In [7]:
from braket.jobs.config import InstanceConfig

instance_config = InstanceConfig(instanceType='ml.m5.large')

`device` 引数を使うと、簡単に異なるデバイスを切り替えることができます。次のセルでは、3 行のうち 1 行のコメントを外すと、Braket ローカルシミュレータ、PennyLane default.qubit シミュレータ、オンデマンドの状態ベクトルシミュレータ SV1 のいずれかを実行できます。

In [8]:
device="local:braket/braket.local.qubit" # Using Braket local simulator
# device="local:pennylane/default.qubit" # Using Pennylane default.qubit simulator
# device="arn:aws:braket:::device/quantum-simulator/amazon/sv1" # Using Braket on-demand SV1 

それでは、ジョブを実行してみましょう！少なくとも、`device`, `source_module`, `entry_point` の引数を指定する必要があります。さらに以下のような他の引数でジョブをカスタマイズすることができます。
- `device`: `"local:<provider>/<simulator_name>"` の形式に従い Embedded Simulator を指定するか、または使用したい Braket オンデマンドシミュレータや QPU の ARN を指定します。これはアルゴリズムスクリプトの環境変数として保存されます。
- `source_module`: アルゴリズムスクリプトを含むファイルまたは Python モジュールへのパスです。Braket Job を実行する際、コンテナにアップロードされます。
- `entry_point`: source_module からの相対パスです。Braket Job の開始時に実行されるコード部分を指します。
- `hyperparameters`: ハイパーパラメータをキーと値で表す Python 辞書です (オプション)。
- `job_name`: ジョブを識別するためのユニークな文字列です。Braket Job コンソールとジョブ ARN に表示されます (オプション)。
- `instance_config`: ジョブ実行時に使用するインスタンスの設定です。デフォルトでは `InstanceConfig(instanceType='ml.m5.large', volumeSizeInGb=30)` です (オプション)。
- `image_uri`: Docker コンテナイメージへのパスです (オプション)。
- `wait_until_complete`: True の場合、関数呼び出しはジョブが完了するまで待機し、さらにローカルコンソールにログを出力します。そうでない場合は、非同期で実行されます。デフォルトは False です (オプション)。

In [9]:
import time
from braket.aws import AwsQuantumJob

job = AwsQuantumJob.create(
    device=device,
    source_module="qaoa",
    entry_point="qaoa.qaoa_algorithm",
    job_name="embedded-simulation-" + str(int(time.time())),
    hyperparameters=hyperparameters,
    instance_config=instance_config,
    image_uri=image_uri,
    wait_until_complete=False,
)

In [10]:
# このセルの実行には 6 分ほどかかります。
print(job.result())

{'params': [[0.9170388720505324, 0.7204929371752304, 1.4528933994269753], [1.3895487869726206, 0.9665264386809151, -0.47009700306676805]], 'cost': -0.5536775832039885}


## 勾配計算方法のカスタマイズ
量子回路の変分パラメータに対するコスト関数の勾配を計算する一般的な方法として、[パラメータシフト法](https://pennylane.ai/qml/glossary/parameter_shift.html)があります。パラメータシフト法では、パラメータをシフトした回路を複数回実行することで、勾配を正確に計算することができます。しかし，SV1 のような並列性の高いシミュレータを用いない限り，シフトされた回路をすべて実行するのは非常に時間がかかります。またその場合でも、回路数はパラメータ数に対して線形に増加します。一方，随伴微分法などの他の勾配法では，必要なメモリ量が増える代わりに，回路の実行回数を少なくすることができます。

例えば、PennyLane の default.qubit シミュレータでは、ハイパーパラメータに `diff_method` 変数を指定することで 随伴微分法を使用することができます。なお、Amazon Braket のオンデマントシミュレータでは、PennyLane のパラメータシフト法しか使えません。

In [11]:
hyperparameters={
    "n_nodes": 6, 
    "n_edges": 8, 
    "n_layers": 3,
    "iterations": 10,
    "stepsize": 0.1,
    "seed": 42,
    "diff_method": "adjoint",
}

In [12]:
job = AwsQuantumJob.create(
    device="local:pennylane/default.qubit",
    source_module="qaoa",
    entry_point="qaoa.qaoa_algorithm",
    job_name="embedded-simulation-" + str(int(time.time())),
    hyperparameters=hyperparameters,
    instance_config=instance_config,
    image_uri=image_uri,
    wait_until_complete=False,
)

In [13]:
# このセルの実行には、6分ほどかかります。
print(job.result())

{'params': [[0.9170388720505318, 0.7204929371752299, 1.452893399426975], [1.3895487869726204, 0.9665264386809148, -0.470097003066768]], 'cost': -0.55367758320399}


## `lightning.gpu` と cuQuantum を用いたシミュレーションの加速
PyTorch と TensorFlow の[ジョブコンテナ](https://docs.aws.amazon.com/braket/latest/developerguide/braket-jobs-script-environment.html?tag=local002-20)には、[NVIDIA cuQuantum ライブラリ](https://developer.nvidia.com/cuquantum-sdk)と PennyLane の [GPU シミュレータ](https://github.com/PennyLaneAI/pennylane-lightning-gpu)である `lightning.gpu` が事前に設定されています。GPU シミュレータは、より大きな回路のシミュレーションを加速し、一定時間内にシミュレーションできる量子ビットの数を増やします。`lightning.gpu` を使用するには、GPU インスタンスを選択する必要があります。Braket Jobs は `lightning.gpu` と互換性のある以下ののインスタンスタイプをサポートしています。
- p3.2xlarge
- p3.8xlarge
- p3.16xlarge

In [14]:
instance_config = InstanceConfig(instanceType='ml.p3.2xlarge')

また、GPU シミュレータは、パラメータシフト法と比較して勾配評価を大幅に高速化できる 随伴微分法もサポートしています。以下では、QAOA を使用して 22 ノードの Max-Clique 問題を解くジョブを作成します。このジョブの実行には 22 量子ビットを必要とします。各最適化にかかる時間は、`lightning.gpu` が約 1 分、CPU ベースのシミュレータである `lightning.qubit` が約 12 分です。この実行時間は、回路サイズ、問題の種類、計算リソースによって異なります。問題やインスタンスが変わると、パフォーマンスの挙動も変わる可能性があります。一般的に、CPU ベースのシミュレータは小さな回路の実行に適しており、GPU ベースのシミュレータは大きな回路に適しています。

In [15]:
hyperparameters={
    "n_nodes": 22, 
    "n_edges": 150, 
    "n_layers": 3,
    "iterations": 1,
    "stepsize": 0.1,
    "seed": 42,
    "diff_method": "adjoint",
}

注意: 次のセルは、デフォルトのリソース制限では完了できない場合があります。アカウントの制限を増やすためには、[AWS サポート](https://support.console.aws.amazon.com/support/home#/case/create?issueType=service-limit-increase)までご連絡ください。

In [18]:
# job = AwsQuantumJob.create(
#     device="local:pennylane/lightning.gpu",
#     source_module="qaoa",
#     entry_point="qaoa.qaoa_algorithm",
#     job_name="embedded-simulation-" + str(int(time.time())),
#     hyperparameters=hyperparameters,
#     instance_config=instance_config,
#     image_uri=image_uri,
#     wait_until_complete=False,
# )

In [19]:
# このセルの実行には、 9 分ほどかかります。
# print(job.result())

## ローカルモードによるデバッグ
ジョブを投入する前に、ローカルでプログラムをデバッグしておくと便利なことがよくあります。Embedded Simulator をローカル環境で実行することで、テストやデバッグを迅速に行うことができます。この機能を使用するには、プログラミング環境に Docker がインストールされている必要があります。 Amazon Braket ノートブックには Docker がプリインストールされていますが、ローカルで、例えばノート PC でコードをテストしたい場合は、Docker をインストールする必要があります。詳細な手順は[こちら](https://docs.docker.com/get-docker/)をご確認ください。

ローカルモードでは、ローカル環境にコンテナが作成され、そのコンテナ上でアルゴリズムが実行されます。ローカルモードでジョブを実行するには、Docker デーモンが起動していることを確認します。Amazon Braket ノートブックインスタンスを使用する場合は、既にこの準備は完了しています。その後、`AwsQuantumJob` の代わりに `LocalQuantumJob` を作成します。ローカルジョブは常に同期して実行され、ログを表示するので、`wait_until_complete` 引数はありません。ローカルモードのジョブはローカル環境で実行されるため、 `instance_config` 引数もありません。ローカルジョブの初回作成時は、コンテナを構築する必要があるため、時間が少しかかりますが、その後の実行はより高速になります。なお、ローカルジョブは、Amazon Braket コンソールには表示されません。ローカルモードでも、量子タスクを実際のデバイスに送信することはできますが、ローカルモード中に実際の QPU に対して実行しても、パフォーマンスのメリットは得られません。ローカルモードの詳細については、[開発者ガイド](https://docs.aws.amazon.com/braket/latest/developerguide/braket-jobs-local-mode.html)を参照してください。

In [20]:
hyperparameters={
    "n_nodes": 6, 
    "n_edges": 8, 
    "n_layers": 3,
    "iterations": 10,
    "stepsize": 0.1,
    "seed": 42,
    "diff_method": "adjoint",
}

In [22]:
from braket.jobs.local.local_job import LocalQuantumJob

# このセルは、初回は3分ほど、その後は30秒ほどで終わります。
job = LocalQuantumJob.create(
    device="local:pennylane/default.qubit",
    source_module="qaoa",
    entry_point="qaoa.qaoa_algorithm",
    job_name="embedded-simulation-" + str(int(time.time())),
    hyperparameters=hyperparameters,
    image_uri=image_uri,
)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Pulling docker container image. This may take a while.


Login Succeeded
1.13.1-gpu-py39-cu117-ubuntu20.04: Pulling from amazon-braket-pytorch-jobs
Digest: sha256:ab518e5af4141a9ce0cb9424c556e21a2fe8c4e4a5f2ff64cfcc85f3d46b3505
Status: Image is up to date for 292282985366.dkr.ecr.us-east-1.amazonaws.com/amazon-braket-pytorch-jobs:1.13.1-gpu-py39-cu117-ubuntu20.04
292282985366.dkr.ecr.us-east-1.amazonaws.com/amazon-braket-pytorch-jobs:1.13.1-gpu-py39-cu117-ubuntu20.04


Using the short-lived AWS credentials found in session. They might expire while running.


Boto3 Version:  1.26.64
Beginning Setup
Checking for Additional Requirements
Additional Requirements Check Finished
Running Code As Subprocess
hyperparams:  {'n_nodes': '6', 'n_edges': '8', 'n_layers': '3', 'iterations': '10', 'stepsize': '0.1', 'seed': '42', 'diff_method': 'adjoint'}
Using local simulator:  Default qubit PennyLane plugin
number of observables:  13
start optimizing...
Metrics - timestamp=1684651535.0420568; Cost=4.570568368761232; iteration_number=0;
Metrics - timestamp=1684651535.30743; Cost=2.6148149684630124; iteration_number=1;
Metrics - timestamp=1684651535.5808728; Cost=1.0421622100067647; iteration_number=2;
Metrics - timestamp=1684651535.845853; Cost=0.286949670853895; iteration_number=3;
Metrics - timestamp=1684651536.109179; Cost=-0.25279908169178755; iteration_number=4;
Metrics - timestamp=1684651536.3736026; Cost=-0.5599800238018436; iteration_number=5;
Metrics - timestamp=1684651536.6450074; Cost=-0.5452012711678209; iteration_number=6;
Metrics - timestamp

## まとめ
このノートブックでは、Hybrid Jobs の Embedded Simulator の使い方を紹介しました。より詳しく知りたい方は、[ドキュメント](https://docs.aws.amazon.com/braket/latest/developerguide/braket-jobs.html)か他のサンプルノートブックをご参照ください。